In [ ]:
%%capture
# Install Unsloth and dependencies
# The %%capture magic prevents installation output from cluttering the notebook
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [2]:
# Verify GPU availability
# Training neural networks requires a GPU for reasonable speeds
# This checks if CUDA (NVIDIA's GPU computing platform) is accessible

import torch

print(f"GPU Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    # BF16 (Brain Float 16) is a special number format that uses less memory while maintaining accuracy
    print(f"BF16 Support: {torch.cuda.is_bf16_supported()}")

GPU Available: True
GPU Name: Tesla T4
GPU Memory: 14.74 GB
BF16 Support: True


In [3]:
from unsloth import FastLanguageModel
import torch

# Configuration parameters
max_seq_length = 2048  # How many tokens (roughly words) the model can see at once
dtype = None  # Let Unsloth choose the best precision format automatically
load_in_4bit = True  # Compress model to save 70% memory - this is Unsloth's magic!

# Load model and tokenizer
# Unsloth's FastLanguageModel automatically optimizes the loading process
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/smollm2-135m",  # A small 135M parameter model, good for learning
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,  # Enables QLoRA (Quantized LoRA)
)

print(f"✓ Model loaded: {model.config._name_or_path}")
print(f"✓ Total parameters: {model.num_parameters():,}")
# 135M parameters means 135 million trainable numbers in the neural network

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.9.0+cu128 with CUDA 1208 (you have 2.8.0+cu126)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.9.0+cu128 with CUDA 1208 (you have 2.8.0+cu126)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✓ Model loaded: unsloth/smollm2-135m
✓ Total parameters: 134,515,584


In [4]:
# Apply LoRA adapters to the model
# Unsloth's get_peft_model automatically patches the model for efficiency
model = FastLanguageModel.get_peft_model(
    model,
    r = 256,  # LoRA rank - higher means more expressive adapters (but more memory)
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers
        "gate_proj", "up_proj", "down_proj",      # Feed-forward layers
        "lm_head", "embed_tokens"                 # Input/output layers (for full fine-tuning)
    ],
    lora_alpha = 256,  # Scaling factor (usually set to rank or 2*rank)
    lora_dropout = 0,  # No dropout for small models
    bias = "none",     # Don't train bias terms (saves memory)
    use_gradient_checkpointing = "unsloth",  # Unsloth's optimized checkpointing (saves 30% memory)
    random_state = 3407,  # Random seed for reproducibility
    use_rslora = False,   # Rank-stabilized LoRA (not needed for this example)
)

# Count how many parameters we're actually training
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = model.num_parameters()

print(f"\n✓ LoRA Applied (Full Fine-tuning)")
print(f"  Trainable params: {trainable_params:,}")
print(f"  Total params: {total_params:,}")
print(f"  Trainable %: {trainable_params/total_params*100:.2f}%")
print(f"  LoRA Rank: 256")
print(f"  LoRA Alpha: 256")

# With high-rank LoRA including embeddings, we train ~45% of parameters
# This is "full fine-tuning" because we're adapting all layers comprehensively

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.11.3 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM

✓ LoRA Applied (Full Fine-tuning)
  Trainable params: 134,775,936
  Total params: 297,603,648
  Trainable %: 45.29%
  LoRA Rank: 256
  LoRA Alpha: 256


In [ ]:
from datasets import load_dataset

# Load a small subset of the CodeParrot dataset
# The dataset is hosted on HuggingFace's dataset hub
print("Loading dataset...")
dataset = load_dataset(
    "codeparrot/codeparrot-clean",  # Clean, deduplicated version
    split="train[:1000]",            # Take only first 1000 examples (out of millions)
    trust_remote_code=True           # Allow custom loading code
)

print(f"\n✓ Dataset loaded: {len(dataset)} samples")
print(f"  Fields: {dataset.column_names}")

# Show a sample to understand the data format
print(f"\nSample code snippet:")
print("-" * 80)
print(dataset[0]['content'][:200])  # Print first 200 characters of code
print("-" * 80)

# Each sample has fields like:
# - 'content': The actual Python code
# - 'repo_name': Which GitHub repository it came from
# - 'path': File path in the repository
# - Various quality metrics (line length, complexity, etc.)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'codeparrot/codeparrot-clean' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'codeparrot/codeparrot-clean' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Loading dataset...


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

file-000000000001.json.gz:   0%|          | 0.00/246M [00:00<?, ?B/s]

...

file-000000000051.json.gz:   0%|          | 0.00/233M [00:00<?, ?B/s]

file-000000000052.json.gz:   0%|          | 0.00/234M [00:00<?, ?B/s]

file-000000000053.json.gz:   0%|          | 0.00/230M [00:00<?, ?B/s]

file-000000000054.json.gz:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer  # Supervised Fine-Tuning Trainer
import os

# Create directory to save model checkpoints
output_dir = "./checkpoints/colab1"
os.makedirs(output_dir, exist_ok=True)

# Configure all training hyperparameters
training_args = TrainingArguments(
    # Batch size configuration
    per_device_train_batch_size = 2,  # How many examples per GPU
    gradient_accumulation_steps = 4,   # Simulate larger batches (2×4=8 effective batch size)

    # Learning rate schedule
    warmup_steps = 10,         # Warm up learning rate gradually
    max_steps = 100,           # Total number of training steps
    learning_rate = 2e-4,      # Step size for parameter updates (0.0002)
    lr_scheduler_type = "linear",  # Gradually decrease learning rate

    # Precision (speed & memory optimization)
    fp16 = not torch.cuda.is_bf16_supported(),  # Use FP16 if BF16 unavailable
    bf16 = torch.cuda.is_bf16_supported(),      # Brain Float 16 (better for training)

    # Optimizer configuration
    optim = "adamw_8bit",     # Unsloth's 8-bit Adam optimizer (saves memory)
    weight_decay = 0.01,      # Prevent overfitting by penalizing large weights

    # Logging and saving
    logging_steps = 5,        # Log metrics every 5 steps
    save_strategy = "steps",  # Save checkpoints periodically
    save_steps = 50,          # Save every 50 steps
    output_dir = output_dir,  # Where to save checkpoints
    report_to = "none",       # Don't use external logging (WandB, TensorBoard)

    # Reproducibility
    seed = 3407,              # Random seed for consistent results
)

print("✓ Training configuration:")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Max steps: {training_args.max_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Optimizer: {training_args.optim}")

In [ ]:
# Define a formatting function
def formatting_func(examples):
    texts = []
    for content in examples["content"]:
        # Format the content as needed for your model
        # For this case, we'll just use the content as is
        texts.append(content)
    return texts # Return the list of strings directly


# Initialize SFTTrainer (Supervised Fine-Tuning Trainer)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "content",  # Field containing code
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # Can make training 5x faster for short sequences
    args = training_args,
    # Removed formatting_func=formatting_func, # Remove the formatting function - Attempting to force update
)

print("\n" + "="*80)
print("STARTING TRAINING - FULL FINE-TUNING")
print("="*80)

# Monitor GPU memory before training
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    print(f"\nGPU Memory before training: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

# Debugging prints
print(f"Type of training_args: {type(training_args)}")
print(f"Value of training_args: {training_args}")
print(f"Type of dataset: {type(dataset)}")
print(f"Length of dataset: {len(dataset)}")
print(f"Dataset columns: {dataset.column_names}")
# Removed the problematic print statement: print(f"Dataset text field: {trainer.dataset_text_field}")

# Explicitly confirm if formatting_func is in trainer arguments (should be False)
print(f"Is 'formatting_func' in trainer arguments? {'formatting_func' in trainer.init_kwargs}")


# Train the model
try:
    trainer_stats = trainer.train()
except TypeError as e:
    print(f"\nCaught a TypeError during training: {e}")
    trainer_stats = None # Ensure trainer_stats is None if an error occurs


# Monitor GPU memory after training
if torch.cuda.is_available():
    print(f"\nGPU Memory after training: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"Peak GPU Memory: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")

print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80)

In [ ]:
# Define a formatting function
def formatting_func(examples):
    texts = []
    for content in examples["content"]:
        # Format the content as needed for your model
        # For this case, we'll just use the content as is
        texts.append(content)
    return texts # Return the list of strings directly


# Initialize SFTTrainer (Supervised Fine-Tuning Trainer)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "content",  # Field containing code
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # Can make training 5x faster for short sequences
    args = training_args,
    formatting_func=formatting_func, # Add the formatting function back
)

print("\n" + "="*80)
print("STARTING TRAINING - FULL FINE-TUNING")
print("="*80)

# Monitor GPU memory before training
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    print(f"\nGPU Memory before training: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

# Train the model
trainer_stats = trainer.train()


# Monitor GPU memory after training
if torch.cuda.is_available():
    print(f"\nGPU Memory after training: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"Peak GPU Memory: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")

print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Extract training logs
logs = trainer.state.log_history
train_logs = [log for log in logs if 'loss' in log]

# Create DataFrame
df = pd.DataFrame(train_logs)
print("\nTraining Statistics:")
print(df[['step', 'loss', 'learning_rate']].to_string(index=False))

# Plot loss curve
if len(df) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(df['step'], df['loss'], marker='o', linewidth=2)
    plt.xlabel('Training Step', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.title('Full Fine-tuning Loss Curve', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/loss_curve.png", dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\n✓ Loss curve saved to {output_dir}/loss_curve.png")

# Print final statistics
print(f"\nFinal Training Statistics:")
print(f"  Total steps: {trainer.state.global_step}")
print(f"  Final loss: {df['loss'].iloc[-1]:.4f}")
print(f"  Average loss: {df['loss'].mean():.4f}")
print(f"  Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"  Samples/second: {trainer_stats.metrics['train_samples_per_second']:.2f}")

In [ ]:
# Enable fast inference mode (2x faster)
FastLanguageModel.for_inference(model)

# Test prompts
test_prompts = [
    "def fibonacci(n):",
    "class DataProcessor:",
    "import numpy as np\n\ndef calculate_mean(",
]

print("\n" + "="*80)
print("CODE GENERATION SAMPLES")
print("="*80)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- Sample {i} ---")
    print(f"Prompt: {prompt}")
    print("\nGenerated Code:")
    print("-" * 80)

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens = 128,
        temperature = 0.7,
        top_p = 0.9,
        do_sample = True,
        use_cache = True,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)
    print("-" * 80)

In [ ]:
# Save LoRA adapter
lora_path = f"{output_dir}/lora_adapter"
model.save_pretrained(lora_path)
tokenizer.save_pretrained(lora_path)
print(f"✓ LoRA adapter saved to {lora_path}")

# Save merged 16-bit model (optional, larger file)
merged_path = f"{output_dir}/merged_16bit"
model.save_pretrained_merged(merged_path, tokenizer, save_method="merged_16bit")
print(f"✓ Merged 16-bit model saved to {merged_path}")

print("\n✓ All checkpoints saved successfully!")